In [1]:
import numpy as np

## Q5
# Inverse of matrix using adjoint method

In [2]:
def determinant(A):
    # calculate determinant custom
    if A.shape == (2,2):
        return A[0,0]*A[1,1] - A[0,1]*A[1,0]
    det = 0
    for i in range(A.shape[0]):
        det += ((-1)**i)*A[0,i]*determinant(A[1:,np.arange(A.shape[1])!=i])
    return det

def inverse_matrix(A):
    # check if matrix is square
    if A.shape[0] != A.shape[1]:
        print("Matrix is not square")
        return None

    # calculate determinant
    det = determinant(A)
    if det == 0:
        print("Matrix is not invertible as det(Matrix) = 0")
        return None
    
    # calculate adjoint
    adj = np.zeros(A.shape)
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            adj[i,j] = ((-1)**(i+j))*determinant(A[np.arange(A.shape[0])!=i][:,np.arange(A.shape[1])!=j])

    # calculate inverse
    A_inv = adj.T/det
    return A_inv

In [7]:
# 3x3 matrix
A = np.array([[-1,3,-2],
              [2,-4,2],
              [0,4,1]])

print("Matrix A:")
print(A)

print("Inverse of A by adjoint method:")
print(inverse_matrix(A))

Matrix A:
[[-1  3 -2]
 [ 2 -4  2]
 [ 0  4  1]]
Inverse of A by adjoint method:
[[ 1.2  1.1  0.2]
 [ 0.2  0.1  0.2]
 [-0.8 -0.4  0.2]]


# Inverse of matrix by row reduction method

In [13]:
def row_swap(A,i,j):
    A[[i,j]] = A[[j,i]]
    return A

def row_addition(A,i,j,k):
    A[j] = A[j] - k*A[i]
    return A

def row_multiplication_scaler(A,i,k):
    A[i] = k*A[i]
    return A

def row_reduction(A):
    # check if matrix is square
    if A.shape[0] != A.shape[1]:
        print("Matrix is not square")
        return None

    # check if matrix is invertible
    if determinant(A) == 0:
        print("Matrix is not invertible as det(Matrix) = 0")
        return None
    
    # do for zero diagonal elements
    for i in range(A.shape[0]):
        if A[i,i] == 0:
            for j in range(i+1,A.shape[0]):
                if A[j,i] != 0:
                    A = row_swap(A,i,j)
                    break

    # augment matrix with identity matrix
    A = np.hstack((A,np.eye(A.shape[0])))

    # perform row operations
    for i in range(A.shape[0]):
        A = row_multiplication_scaler(A,i,1/A[i,i])
        for j in range(A.shape[0]):
            if i != j:
                A = row_addition(A,i,j,A[j,i])

    # extract inverse matrix
    A_inv = A[:,A.shape[0]:]
    return A_inv

In [14]:
# 3x3 matrix
A = np.array([[-1,3,-2],
              [2,-4,2],
              [0,4,1]])

print("Matrix A:")
print(A)

print("Inverse of A by row reduction method:")
print(row_reduction(A))


Matrix A:
[[-1  3 -2]
 [ 2 -4  2]
 [ 0  4  1]]
Inverse of A by row reduction method:
[[ 1.2  1.1  0.2]
 [ 0.2  0.1  0.2]
 [-0.8 -0.4  0.2]]


## Q8

![./Q7.png](./Q7.png)

In [2]:
# compute element stiffness matrix, given k
def element_stiffness(k):
    Ke = k*np.array([
        [1, -1],
        [-1, 1]
    ])
    return Ke

# assemble stiffness matrix
def computeStiffnessMatrix(e_n, K, Ke):
    if (e_n.ndim == 1):
        idx = e_n - 1
        K[np.ix_(idx, idx)] += Ke
    else:
        for row in e_n:
            idx = row - 1
            K[np.ix_(idx, idx)] += Ke
    return K

# solve eq, considering boundary conditions of u
def get_disp_reac(K, f, bc_zero=None, bc_cust=None):
    nD = K.shape[0]
    n_bc = bc_zero.shape[0]

    if bc_cust is None:
        bc_cust = np.zeros([n_bc])

    if bc_zero is None:
        return np.linalg.solve(K, f)

    bc = np.ones(nD, 'bool')
    bcD = np.arange(nD)

    bc[np.ix_(bc_zero-1)] = False
    bcD = bcD[bc]

    fS = f[bcD] - K[np.ix_((bcD), (bc_zero-1))] * np.asmatrix(bc_cust.reshape(n_bc, 1))

    aS = np.linalg.solve(K[np.ix_((bcD), (bcD))], fS)

    a = np.zeros([nD, 1])
    a[np.ix_(bc_zero-1)] = np.asmatrix(bc_cust.reshape(n_bc, 1))
    a[np.ix_(bcD)] = aS

    Q = K*np.asmatrix(a) - f

    return np.asmatrix(a), Q


# get element displacement -> no need
def get_element_disp(e_n, a):
    ed = None
    if (e_n.ndim==1):
        nD = len(e_n)
        ed = np.zeros([nD])
        idx = e_n - 1
        ed[:] = a[np.ix_(idx)].T
    else:
        nE = e_n.shape[0]
        nD = e_n.shape[1]
        eD = np.zeros([nE, nD])
        for i, row in enumerate(e_n):
            idx = row - 1
            eD[i, :] = a[np.ix_(idx)].T
    
    return ed

# get element force
def get_element_force(ek, eu):
    k = ek

    F = k*(eu[1] - eu[0])
    return F


In [3]:
a = np.zeros([3])
type(a[0])

numpy.float64

In [4]:
# Adding nodes and elements
E_n = np.array([
    [1, 3],
    [2, 3],
    [3, 4]
])

# stiffness matrix and load f
K = np.zeros((4, 4))
f = np.zeros((4, 1))

# element stiffness matrix
k = 500.
k1 = k
k2 = 2*k

# for k1
Ke1 = element_stiffness(k1)
Ke2 = element_stiffness(k2)

# assemble stiffness matrix
computeStiffnessMatrix(E_n[0, :], K, Ke1)
computeStiffnessMatrix(E_n[1, :], K, Ke1)
computeStiffnessMatrix(E_n[2, :], K, Ke2)

# boundary conditions
bc = np.array([1, 2, 4])
f[2] = 4

# solve
a, Q = get_disp_reac(K, f, bc)

print("Displacement:")
print(a)

print("Reaction:")
print(Q)

# get element dist
ed1 = get_element_disp(E_n[0, :], a)
ed2 = get_element_disp(E_n[1, :], a)
ed3 = get_element_disp(E_n[2, :], a)


ef1 = get_element_force(k1, ed1)
ef2 = get_element_force(k1, ed2)
ef3 = get_element_force(k2, ed3)

print("Element force 1:")
print(ef1)

print("Element force 2:")
print(ef2)

print("Element force 3:")
print(ef3)


Displacement:
[[0.   ]
 [0.   ]
 [0.002]
 [0.   ]]
Reaction:
[[-1.]
 [-1.]
 [ 0.]
 [-2.]]
Element force 1:
1.0
Element force 2:
1.0
Element force 3:
-2.0


In [25]:
aS

matrix([[0.002]])

In [51]:
# final 
def custum_linagSolve(A, B):
    '''
    Solve linear equation Ax = B
    '''
    n = A.shape[0]
    x = np.zeros([n, 1])
    for i in range(n):
        x[i] = (B[i] - np.sum(A[i, np.arange(n)!=i]*x[np.arange(n)!=i]))/A[i,i]
    return x

def oneSpringStiffness(k):
    '''
    Return 2x2 stiffness matrix for element
    '''
    K = k*np.array([
        [1, -1],
        [-1, 1]
    ])
    return K

def computeStiffnessSystem(Spring_nodes ,K , Ke):
    '''
    Assemble the global stiffness matrix
    '''
    if (Spring_nodes.ndim == 1):
        idx = Spring_nodes - 1
        K[np.ix_(idx, idx)] += Ke
    else:
        for row in Spring_nodes:
            idx = row - 1
            K[np.ix_(idx, idx)] += Ke
    return K

# get global force and displacement
def get_force_disp(K, f, bc_zero = None, bc_cust = None):
    '''
    Solve the system of equations to get global reaction force and displacement
    '''
    n_nodes = K.shape[0]
    n_nodesBC = bc_zero.shape[0]

    if bc_cust is None:
        bc_cust = np.zeros([n_nodesBC])

    if bc_zero is None:
        return np.linalg.solve(K, f)
    
    bc_idx = np.ones(n_nodes, 'bool')
    bc_disp = np.arange(n_nodes)

    bc_idx[np.ix_(bc_zero-1)] = False
    bc_disp = bc_disp[bc_idx]

    fSys = f[bc_disp] - K[np.ix_(bc_disp, bc_zero-1)] * np.asmatrix(bc_cust.reshape(n_nodesBC, 1))
    uSys = np.linalg.solve(K[np.ix_(bc_disp, bc_disp)], fSys)
    # uSys = custum_linagSolve(K[np.ix_(bc_disp, bc_disp)], fSys)

    u = np.zeros([n_nodes, 1])
    u[np.ix_(bc_zero-1)] = np.asmatrix(bc_cust.reshape(n_nodesBC, 1))
    u[np.ix_(bc_disp)] = uSys

    Fsys = K*np.asmatrix(u) - f

    return np.asmatrix(u), Fsys


In [52]:
# Adding nodes and elements
E_n = np.array([
    [1, 3],
    [2, 3],
    [3, 4]
])

# stiffness matrix and load f
K = np.zeros((4, 4))
f = np.zeros((4, 1))

# element stiffness matrix
k = 500.
k1 = k
k2 = k
k3 = 2*k

# for k1
Ke1 = oneSpringStiffness(k1)
Ke2 = oneSpringStiffness(k2)
Ke3 = oneSpringStiffness(k3)


# assemble stiffness matrix
computeStiffnessSystem(E_n[0, :], K, Ke1)
computeStiffnessSystem(E_n[1, :], K, Ke2)
computeStiffnessSystem(E_n[2, :], K, Ke3)

# print the stiffness matrix
print("Stiffness Matrix:")
print(K)

# boundary conditions
bc = np.array([1, 2, 4])
f[2] = 4

# solve
u, F = get_force_disp(K, f, bc)

print("Displacement:")
print(u)

print("Reaction:")
print(F)

Stiffness Matrix:
[[  500.     0.  -500.     0.]
 [    0.   500.  -500.     0.]
 [ -500.  -500.  2000. -1000.]
 [    0.     0. -1000.  1000.]]
Displacement:
[[0.   ]
 [0.   ]
 [0.002]
 [0.   ]]
Reaction:
[[-1.]
 [-1.]
 [ 0.]
 [-2.]]
